In [24]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from sklearn.externals import joblib
%matplotlib inline

In [25]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Get hog features
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    features = hog(img, orientations=orient, 
                    pixels_per_cell=(pix_per_cell, pix_per_cell),
                    cells_per_block=(cell_per_block, cell_per_block),
                    block_norm= 'L2-Hys',
                    transform_sqrt=True, 
                    visualise=vis, feature_vector=feature_vec)
    return features


# get hog features for all image channels
def get_hog_feature_all_channels(img, orient, pix_per_cell, cell_per_block):
    ch1 = img[:,:,0]
    ch2 = img[:,:,1]
    ch3 = img[:,:,2]
    hf_ch1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
    hf_ch2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
    hf_ch3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False)
    return hf_ch1, hf_ch2, hf_ch3

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', include_spatial=True, spatial_size=(32, 32),
                     include_hist=True, hist_bins=32, hist_range=(0, 256),
                     include_hog=True, orient=11, pix_per_cell=8, cell_per_block=3, feature_vec=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        img = cv2.imread(file)
        img_i = np.invert(img)
        for image in [img]:
            img_features = []
            # apply color conversion if other than 'RGB'
            if cspace != 'RGB':
                if cspace == 'HSV':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                elif cspace == 'LUV':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2Luv)
                elif cspace == 'HLS':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
                elif cspace == 'YUV':
                    feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
            else:
                feature_image = np.copy(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            # Apply bin_spatial() to get spatial color features
            if include_spatial:
                spatial_features = bin_spatial(feature_image, size=spatial_size)
                img_features.append(spatial_features)
            # Apply color_hist() also with a color space option now
            if include_hist:
                hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
                img_features.append(hist_features)
            # include hog features
            if include_hog:
                hf1, hf2, hf3 = get_hog_feature_all_channels(feature_image, orient, pix_per_cell, cell_per_block)
                '''hog_features, hog_img = get_hog_features(feature_image[:,:,0], orient, pix_per_cell, cell_per_block,
                                                vis=True, feature_vec=feature_vec)'''
                hog_features = np.hstack((hf1.ravel(), hf2.ravel(), hf3.ravel()))
                img_features.append(hog_features)
            #print(spatial_features.shape, hist_features.shape, hog_features.shape, feature_image.shape, hf1.shape)
            # Append the new feature vector to the features list
            features.append(np.concatenate(img_features))
    # Return list of feature vectors
    return features

In [66]:
# Read in car and non-car images
ext = 'jpeg'
car_images = [glob.glob('./train/vss/0/*.'+ext), glob.glob('./train/vss/1/*.'+ext), glob.glob('./train/vss/2/*.'+ext),
              glob.glob('./train/vss/3/*.'+ext), glob.glob('./train/vss/4/*.'+ext), glob.glob('./train/vss/5/*.'+ext)]
notcar_images = glob.glob('./train/nvs/all/*.'+ext)
cars = []
notcars = []
for image_dir in car_images:
    for image in image_dir:
        cars.append(image)
for image in notcar_images:
    notcars.append(image)

# TODO play with these values to see how your classifier
# performs under different binning scenarios
spatial = 16
histbin = 32
color_space = 'YUV'
orient = 11
pix_per_cell = 8
cell_per_block = 2
print("Extracting features..")
t0=time.time()
car_features = extract_features(cars, cspace=color_space, spatial_size=(spatial, spatial), include_hist=True,
                        hist_bins=histbin, hist_range=(0, 256),include_hog=True, include_spatial=True,
                        orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, feature_vec=True)
notcar_features = extract_features(notcars, cspace=color_space, spatial_size=(spatial, spatial), include_hist=True,
                        hist_bins=histbin, hist_range=(0, 256),include_hog=True, include_spatial=True,
                        orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, feature_vec=True)
t1=time.time()
print(round(t1-t0, 2), 'Seconds to extract features')
print(len(car_features), len(notcar_features))
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

Extracting features..
12.16 Seconds to extract features
1196 1125


In [74]:
# Split up data into randomized training and test sets
#rand_state = np.random.randint(0, 100)
print(len(X), len(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)#, random_state=rand_state)
print(X_train.shape)
# Fit a per-column scaler only on the training data
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X_train and X_test
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using spatial binning of:',spatial,
    'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
print("Training..")
# Use a linear SVC 
svc = LinearSVC()
#svc = SVC(C=10, gamma=1000, kernel='rbf')
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = len(y_test)
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 2), 'Seconds to predict', n_predict,'labels with SVC')

print("Saving the model..")
joblib.dump(svc, 'svm_model.pkl')
joblib.dump(X_scaler, 'svm_scaler.pkl')
print("Model saved.")

2321 2321
(1972, 7332)
Using spatial binning of: 16 and 32 histogram bins
Feature vector length: 7332
Training..
0.42 Seconds to train SVC...
Test Accuracy of SVC =  0.9971
My SVC predicts:  [1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1.
 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0.
 1. 0.